## mwe about dofs and 

In [235]:
import ufl
from mpi4py import MPI
from petsc4py import PETSc
from petsc4py.PETSc import ScalarType
#from petsc4py.PETSc import VectorType

import dolfinx
import numpy as np

mesh = dolfinx.mesh.create_rectangle(MPI.COMM_WORLD,[[0,0],[1,1]], [10, 10])

element = ufl.VectorElement('Lagrange',mesh.ufl_cell(),degree=1,dim=2)
V = dolfinx.fem.FunctionSpace(mesh, element)

def bottom(x):
    return np.isclose(x[1], 0.0)

V_y = V.sub(1).collapse()[0]

blocked_dofs_bottom = dolfinx.fem.locate_dofs_geometrical((V.sub(1), V_y), bottom)

zero_uy = dolfinx.fem.Function(V_y)

with zero_uy.vector.localForm() as bc_local:
    bc_local.set(0.0)

## method 1 using dofs

In [35]:
# Import required libraries
import matplotlib.pyplot as plt
import numpy as np
from dolfinx import *
from ufl import *
from mpi4py import *
from petsc4py import *

from dolfinx.fem import *
from dolfinx.fem.petsc import *
from dolfinx.mesh import *
from petsc4py.PETSc import *

plt.rcParams["figure.figsize"] = (40,6)

In [46]:
import meshio
from dolfinx import io
from mpi4py import MPI

def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells})
    return out_mesh

meshy = meshio.read("/mnt/d/Research Projects/FEniCS/crack/3d.msh")
print(meshy.cells_dict.keys())

meshio.write("t3d.xdmf", create_mesh(meshy, "tetra", False))

# Define parameters
meshfile = "t3d.xdmf"

# Read the mesh
with io.XDMFFile(MPI.COMM_WORLD, meshfile, "r") as file:
    mesh = file.read_mesh(name='Grid')


dict_keys(['tetra'])


In [37]:
# # import gmsh mesh using meshio
# import meshio
# from dolfinx import io
# from mpi4py import MPI

# def create_mesh(mesh, cell_type, prune_z=False):
#     cells = mesh.get_cells_type(cell_type)
#     points = mesh.points[:,:2] if prune_z else mesh.points
#     out_mesh = meshio.Mesh(points=points, cells={cell_type: cells})
#     return out_mesh

# meshy = meshio.read("/mnt/d/Research Projects/FEniCS/crack/t4.msh")
# meshio.write("t4.xdmf", create_mesh(meshy, "triangle", True))

# # Define parameters
# meshfile = "t4.xdmf"

# # Read the mesh
# with io.XDMFFile(MPI.COMM_WORLD, meshfile, "r") as file:
#     mesh = file.read_mesh(name='Grid')

In [38]:
# mesh = mesh.create_rectangle(MPI.COMM_WORLD,[[0,0],[1,1]], [10, 10])

In [39]:
element = VectorElement('CG',mesh.ufl_cell(),degree=1,dim=2)
V = FunctionSpace(mesh, element)

In [40]:
def bottom_no_crack(x):
    return np.logical_and(np.isclose(x[1], 0.0), x[0] > 0.3)

def right(x):
    return np.isclose(x[0], 1.)

V_x = V.sub(0).collapse()[0]
V_y = V.sub(1).collapse()[0]

blocked_dofs_bottom = locate_dofs_geometrical((V.sub(1), V_y), bottom_no_crack)
blocked_dofs_right = locate_dofs_geometrical((V.sub(0), V_x), right)

In [41]:
zero_uy = Function(V_y)
with zero_uy.vector.localForm() as bc_local:
    bc_local.set(0.0)

zero_ux = fem.Function(V_x)
with zero_ux.vector.localForm() as bc_local:
    bc_local.set(0.0)
      
bc0 = fem.dirichletbc(zero_uy, blocked_dofs_bottom, V.sub(1))
bc1 = fem.dirichletbc(zero_ux, blocked_dofs_right, V.sub(0))
bcs = [bc0, bc1]

In [42]:
from dolfinx.mesh import *

dx = Measure("dx",domain=mesh)
top_facets = locate_entities_boundary(mesh, 1, lambda x : np.isclose(x[1], 1.))
mt = meshtags(mesh, 1, top_facets, 1)
ds = Measure("ds", subdomain_data=mt)

In [43]:
from petsc4py.PETSc import *

u = TrialFunction(V)
v = TestFunction(V)

E = 1. 
nu = 0.3 
mu = E / (2.0 * (1.0 + nu))
lmbda = E * nu / ((1.0 + nu) * (1.0 - 2.0 * nu))

# this is for plane-stress
# lmbda = 2*mu*lmbda/(lmbda+2*mu)

def eps(u):
    """Strain"""
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)

def sigma(u):
    """Stress"""
    return lmbda * nabla_div(u) * Identity(len(u)) + 2*mu*eps(u)

def a(u,v):
    """The bilinear form of the weak formulation"""
    return inner(sigma(u), eps(v)) * dx 

def L(v): 
    """The linear form of the weak formulation"""
    # Volume force
    b = fem.Constant(mesh,ScalarType((0,0,0)))

    # Surface force on the top
    f = fem.Constant(mesh,ScalarType((0,0.1,0)))
    return dot(b, v) * dx + dot(f, v) * ds(1)

In [44]:
print(grad(u).ufl_shape)

(2, 3)


In [45]:
problem = fem.petsc.LinearProblem(a(u,v), L(v), bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()
uh.name = "Displacement"

Can't add expressions with different shapes.


ERROR:UFL:Can't add expressions with different shapes.


UFLException: Can't add expressions with different shapes.

In [12]:
with io.XDMFFile(MPI.COMM_WORLD, "disp.xdmf", "w") as file:
        file.write_mesh(uh.function_space.mesh)
        file.write_function(uh)

In [13]:
W=fem.TensorFunctionSpace(mesh,("DG",0))
sigma=fem.Function(W)
sigma_exp= fem.Expression(lmbda * tr(eps(uh))* Identity(len(u))  + 2*mu*sym(grad(uh).T),W.element.interpolation_points())
sigma.interpolate(sigma_exp)

with io.XDMFFile(MPI.COMM_WORLD, "stress.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    sigma.name = "sigma"
    xdmf.write_function(sigma)

In [14]:
s = lmbda * tr(eps(uh))* Identity(len(u))  + 2*mu*sym(grad(uh).T) - 1./3*tr(lmbda * tr(eps(uh))* Identity(len(u))  + 2*mu*sym(grad(uh).T))*Identity(len(uh))
von_Mises = sqrt(3./2*inner(s, s))

V_von_mises = fem.FunctionSpace(mesh, ("DG", 0))
stress_expr = fem.Expression(von_Mises, V_von_mises.element.interpolation_points())
stresses = fem.Function(V_von_mises)
stresses.interpolate(stress_expr)

with io.XDMFFile(MPI.COMM_WORLD, "vonmises_stress.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_function(stresses)